## 1\. Customer Location Analysis

**Proposition:** Identify the top 5 cities with the most customers.

**Functional Specification:**

- Use `Application.Cities` and `Sales.Customers` tables.
- Count customers in each city.
- Rank cities by customer count.
- Display the top 5 cities.

In [1]:
USE WideWorldImporters;
GO

SELECT TOP 5
    c.CityName,
    COUNT(cust.CustomerID) AS CustomerCount
FROM 
    Application.Cities c
INNER JOIN 
    Sales.Customers cust ON c.CityID = cust.DeliveryCityID
GROUP BY 
    c.CityID, c.CityName
ORDER BY 
    CustomerCount DESC

Commands completed successfully.

(5 rows affected)

Total execution time: 00:00:00.038

CityName,CustomerCount
Akhiok,2
Cherry Grove Beach,2
Isabela,2
East Fultonham,2
Palmas del Mar,2


## 2\. Popular Product Categories

**Proposition:** Determine the 3 most popular product categories by sales volume.

**Functional Specification:**

- Use `Warehouse.StockGroups` and `Sales.OrderLines` tables.
- Sum the quantity sold for each stock group.
- Rank stock groups by total quantity sold.
- Show the top 3 categories.

In [2]:
SELECT TOP 3
    sg.StockGroupName,
    SUM(ol.Quantity) AS TotalQuantitySold
FROM 
    Warehouse.StockGroups sg
INNER JOIN 
    Warehouse.StockItemStockGroups sisg ON sg.StockGroupID = sisg.StockGroupID
INNER JOIN 
    Warehouse.StockItems si ON sisg.StockItemID = si.StockItemID
INNER JOIN 
    Sales.OrderLines ol ON si.StockItemID = ol.StockItemID
GROUP BY 
    sg.StockGroupID, sg.StockGroupName
ORDER BY 
    TotalQuantitySold DESC

(3 rows affected)

Total execution time: 00:00:00.096

StockGroupName,TotalQuantitySold
Packaging Materials,5838043
Clothing,2842918
Computing Novelties,2181299


## 3\. Monthly Sales Trend

**Proposition:** Calculate total sales for each month in the most recent year.

**Functional Specification:**

- Use `Sales.Orders` and `Sales.OrderLines` tables.
- Sum the total sales for each month.
- Display results in chronological order.

In [3]:
WITH RecentYear AS (
    SELECT TOP 1 YEAR(OrderDate) AS Year
    FROM Sales.Orders
    ORDER BY Year DESC
)
SELECT 
    DATENAME(MONTH, o.OrderDate) AS Month,
    MONTH(o.OrderDate) AS MonthNumber,
    SUM(ol.Quantity * ol.UnitPrice) AS TotalSales
FROM 
    Sales.Orders o
INNER JOIN 
    Sales.OrderLines ol ON o.OrderID = ol.OrderID
CROSS JOIN
    RecentYear
WHERE 
    YEAR(o.OrderDate) = RecentYear.Year
GROUP BY 
    DATENAME(MONTH, o.OrderDate),
    MONTH(o.OrderDate)
ORDER BY 
    MonthNumber

(5 rows affected)

Total execution time: 00:00:00.299

Month,MonthNumber,TotalSales
January,1,4612140.45
February,2,4099480.35
March,3,4807110.70
April,4,4739058.60
May,5,5138002.65


## 4\. Top 10 Customers

**Proposition:** Identify the top 10 customers by total purchase amount.

**Functional Specification:**

- Use `Sales.Customers` and `Sales.Orders` tables.
- Calculate total purchase amount for each customer.
- Rank customers by total purchase amount.
- List the top 10 customers.

In [4]:
SELECT TOP 10
    c.CustomerName,
    SUM(ol.Quantity * ol.UnitPrice) AS TotalPurchaseAmount
FROM 
    Sales.Customers c
INNER JOIN 
    Sales.Orders o ON c.CustomerID = o.CustomerID
INNER JOIN 
    Sales.OrderLines ol ON o.OrderID = ol.OrderID
GROUP BY 
    c.CustomerID, c.CustomerName
ORDER BY 
    TotalPurchaseAmount DESC

(10 rows affected)

Total execution time: 00:00:00.107

CustomerName,TotalPurchaseAmount
"Tailspin Toys (Inguadona, MN)",384393.35
"Tailspin Toys (Minidoka, ID)",379660.70
Mauno Laurila,377189.80
"Wingtip Toys (Sarversville, PA)",372350.00
Ingrida Zeltina,368067.45
"Tailspin Toys (Long Meadow, MD)",367258.50
Nasrin Omidzadeh,366883.75
"Wingtip Toys (Cuyamungue, NM)",365915.45
"Wingtip Toys (San Jacinto, CA)",365330.95
"Wingtip Toys (Morrison Bluff, AR)",360652.80


## 5\. Average Order Fulfillment Time

**Proposition:** Calculate the average time between order placement and delivery.

**Functional Specification:**

- Use `Sales.Orders` table.
- Calculate the difference between `OrderDate` and `ExpectedDeliveryDate`.
- Compute the average of these differences.

In [ ]:
SELECT 
    AVG(DATEDIFF(day, OrderDate, ExpectedDeliveryDate)) AS AverageFulfillmentDays
FROM 
    Sales.Orders
WHERE
    OrderDate IS NOT NULL
    AND ExpectedDeliveryDate IS NOT NULL
    AND ExpectedDeliveryDate >= OrderDate

## 6\. Supplier Performance

**Proposition:** List the top 5 suppliers by on-time delivery percentage.

**Functional Specification:**

- Use `Purchasing.Suppliers` and `Purchasing.PurchaseOrders` tables.
- Calculate the percentage of on-time deliveries for each supplier.
- Rank suppliers by this percentage.
- Show the top 5 suppliers.

In [5]:
WITH SupplierDeliveries AS (
    SELECT 
        s.SupplierID,
        s.SupplierName,
        COUNT(*) AS TotalOrders,
        SUM(CASE 
            WHEN po.ExpectedDeliveryDate >= po.OrderDate 
                AND (po.DeliveryMethodID IS NOT NULL OR po.OrderDate <= GETDATE())
            THEN 1 ELSE 0 
        END) AS OnTimeDeliveries
    FROM 
        Purchasing.Suppliers s
    JOIN 
        Purchasing.PurchaseOrders po ON s.SupplierID = po.SupplierID
    GROUP BY 
        s.SupplierID, s.SupplierName
)
SELECT TOP 5
    SupplierName,
    TotalOrders,
    OnTimeDeliveries,
    CAST(OnTimeDeliveries AS FLOAT) / NULLIF(TotalOrders, 0) * 100 AS OnTimeDeliveryPercentage
FROM 
    SupplierDeliveries
WHERE 
    TotalOrders > 0
ORDER BY 
    OnTimeDeliveryPercentage DESC

(5 rows affected)

Total execution time: 00:00:00.084

SupplierName,TotalOrders,OnTimeDeliveries,OnTimeDeliveryPercentage
Graphic Design Institute,13,13,100
"Fabrikam, Inc.",1055,1055,100
Northwind Electric Cars,10,10,100
A Datum Corporation,5,5,100
"Litware, Inc.",985,985,100


## 7\. Average Product Price by Category

**Proposition:** Calculate the average price of products in each category (stock group).

**Functional Specification:**

- Use `Warehouse.StockItems` and `Warehouse.StockGroups` tables.
- Calculate the average unit price for products in each stock group.
- Display results sorted by average price in descending order.

In [7]:
SELECT 
    sg.StockGroupName,
    AVG(si.UnitPrice) AS AveragePrice,
    COUNT(si.StockItemID) AS NumberOfProducts
FROM 
    Warehouse.StockItems si
JOIN 
    Warehouse.StockItemStockGroups sisg ON si.StockItemID = sisg.StockItemID
JOIN 
    Warehouse.StockGroups sg ON sisg.StockGroupID = sg.StockGroupID
GROUP BY 
    sg.StockGroupName
ORDER BY 
    AveragePrice DESC

(9 rows affected)

Total execution time: 00:00:00.034

StockGroupName,AveragePrice,NumberOfProducts
Toys,123.714285,21
Packaging Materials,62.203880,67
USB Novelties,60.714285,14
Novelty Items,45.965384,91
Furry Footwear,27.500000,24
Clothing,23.905405,74
Computing Novelties,22.680722,83
T-Shirts,18.000000,26
Mugs,13.000000,42


## 8\. Top Selling Products Analysis

**Proposition:** <span style="color: rgb(0, 0, 0); font-family: -webkit-standard; font-size: medium; caret-color: rgb(0, 0, 0);">Identify the top 5 best-selling products by total sales value.</span>

**Functional Specification:**

- Use `Sales.OrderLines` and `Warehouse.StockItems` tables.
- Calculate the total sales value for each product.
- Rank products by total sales value.
- Display the top 5 products with their names and total sales value.

In [15]:
SELECT TOP 5
    si.StockItemName,
    SUM(ol.Quantity * ol.UnitPrice) AS TotalSalesValue
FROM 
    Sales.OrderLines ol
JOIN 
    Warehouse.StockItems si ON ol.StockItemID = si.StockItemID
GROUP BY 
    si.StockItemID, si.StockItemName
ORDER BY 
    TotalSalesValue DESC

(5 rows affected)

Total execution time: 00:00:00.042

StockItemName,TotalSalesValue
Air cushion machine (Blue),11107251.00
32 mm Anti static bubble wrap (Blue) 50m,6384000.00
10 mm Anti static bubble wrap (Blue) 50m,6329070.00
20 mm Double sided bubble wrap 50m,6214320.00
32 mm Double sided bubble wrap 50m,6190240.00


## 9\. Popular Payment Method

**Proposition:** Determine the most commonly used payment method.

**Functional Specification:**

- Use `Application.PaymentMethods` and `Sales.CustomerTransactions` tables.
- Count the usage of each payment method.
- Rank payment methods by usage count.
- Show the top payment method.

In [10]:
SELECT TOP 1
    pm.PaymentMethodName,
    COUNT(ct.CustomerTransactionID) AS UsageCount
FROM 
    Application.PaymentMethods pm
LEFT JOIN 
    Sales.CustomerTransactions ct ON pm.PaymentMethodID = ct.PaymentMethodID
GROUP BY 
    pm.PaymentMethodID, pm.PaymentMethodName
ORDER BY 
    UsageCount DESC

(1 row affected)

Total execution time: 00:00:00.062

PaymentMethodName,UsageCount
EFT,26637


## 10\. Top 5 Selling Products

**Proposition:** Identify the top 5 selling products by total quantity sold.

**Functional Specification:**

- Use `Sales.OrderLines` and <span style="font-size: 12px;">Warehouse.StockItems</span> tables.
- Sum the tital quantity sold for each product.
- Rank products by total quantity sold.
- Display prodcts by total quanity sold.
- Display the top 5 products with their names and total quantity sold.

In [9]:
SELECT TOP 5
    si.StockItemName,
    SUM(ol.Quantity) AS TotalQuantitySold
FROM 
    Sales.OrderLines ol
JOIN 
    Warehouse.StockItems si ON ol.StockItemID = si.StockItemID
GROUP BY 
    si.StockItemID, si.StockItemName
ORDER BY 
    TotalQuantitySold DESC

(5 rows affected)

Total execution time: 00:00:00.037

StockItemName,TotalQuantitySold
Black and orange fragile despatch tape 48mmx75m,207324
Black and orange fragile despatch tape 48mmx100m,193680
Clear packaging tape 48mmx75m,158626
3 kg Courier post bag (White) 300x190x95mm,152375
Shipping carton (Brown) 356x356x279mm,152125
